# Wstęp do uczenia maszynowego - praca domowa nr 5

#### Jędrzej Sokołowski, Filip Szympliński
#### 9 maja 2022

### Wczytanie pakietów oraz danych

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import copy
warnings.filterwarnings('ignore')

from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score


# ustawia domyślną wielkość wykresów
plt.rcParams['figure.figsize'] = (8,8)
# to samo tylko dla tekstu
plt.rcParams['font.size'] = 16
# ustawia wielkość tekstów dla wykresów seaborn zależną od wielkości wykresu
sns.set_context('paper', font_scale=1.4)

In [ ]:
data = pd.read_csv("/Users/danieltytkowski/Downloads/urbanGB/urbanGB.txt", sep=",", names=["x","y"])

### Szybkie spojrzenie na dane

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
data["x"] = data["x"]/1.7

Zgodnie z zaleceniem, które było w pliku README.md, skalujemu odpowiednio długość geograficzną punktów, aby ta zmienna lepiej odwzorywała odległości w stosunku do szerokości geograficznej.

In [ ]:
X = data["x"].values
Y = data["y"].values

In [ ]:
data_array = data.to_numpy()

In [ ]:
plt.scatter(X, Y)
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score
from scipy.spatial import distance


# Single output functions

def min_interclust_dist(X, label):
    clusters = set(label)
    # X = X.to_numpy()
    global_min_dist = np.inf
    for cluster_i in clusters:
        cluster_i_idx = np.where(label == cluster_i)
        for cluster_j in clusters:
            if cluster_i != cluster_j:
                cluster_j_idx = np.where(label == cluster_j)
                interclust_min_dist = np.min(distance.cdist(X[cluster_i_idx], X[cluster_j_idx]))
                global_min_dist = np.min([global_min_dist, interclust_min_dist])
    return global_min_dist

def _inclust_mean_dists(X, label):
    clusters = set(label)
    # X = X.to_numpy()
    inclust_dist_list = []
    for cluster_i in clusters:
        cluster_i_idx = np.where(label == cluster_i)
        inclust_dist = np.mean(distance.pdist(X[cluster_i_idx]))
        inclust_dist_list.append(inclust_dist)
    return inclust_dist_list

def mean_inclust_dist(X, label):
    # X = X.to_numpy()
    inclust_dist_list = _inclust_mean_dists(X, label)
    return np.mean(inclust_dist_list)

def std_dev_of_inclust_dist(X, label):
    # X = X.to_numpy()
    inclust_dist_list = _inclust_mean_dists(X, label)
    return np.std(inclust_dist_list)

def mean_dist_to_center(X, label):
    clusters = set(label)
    # X = X.to_numpy()
    inclust_dist_list = []
    for cluster_i in clusters:
        cluster_i_idx = np.where(label == cluster_i)
        cluster_i_mean = np.mean(X[cluster_i_idx], axis=0, keepdims=True)
        inclust_dist = np.mean(distance.cdist(X[cluster_i_idx], cluster_i_mean))
        inclust_dist_list.append(inclust_dist)
    return np.mean(inclust_dist_list)


metric_function_list = [mean_dist_to_center, std_dev_of_inclust_dist, mean_inclust_dist, min_interclust_dist]

# Plot scores function
def count_clustering_scores(X, cluster_num, model, score_fun):

    if isinstance(cluster_num, int):
        cluster_num_iter = [cluster_num]
    else:
        cluster_num_iter = cluster_num
        
    scores = []    
    for k in cluster_num_iter:
        model_instance = model(n_clusters=k)
        labels = model_instance.fit_predict(X)
        wcss = score_fun(X, labels)
        scores.append(wcss)
    
    if isinstance(cluster_num, int):
        return scores[0]
    else:
        return scores

# All single output metrics function
def print_metrics_results(X, model, n_cls, linkage_method):

    model_instance = model(n_clusters=n_cls, linkage=linkage_method)
    model_instance.fit(X)
    y_pred= model_instance.predict(data)
    plt.scatter(data[:, 0], data[:, 1], c=y_pred, s=30, cmap='plasma')
    plt.show()

    print(f"Davies Bouldin Score: {davies_bouldin_score(X, y_pred)}")
    print(f"Calinski Harabasz Score: {calinski_harabash_score(X, y_pred)}")
    print(f'Minimal distance between clusters = {count_clustering_scores(X, n_cls, model, min_interclust_dist):.2f}.')
    print(f'Average distance between points in the same class = '
      f'{count_clustering_scores(X, n_cls, model, mean_inclust_dist):.2f}.')
    print(f'Standard deviation of distance between points in the same class = '
      f'{count_clustering_scores(X, n_cls, model, std_dev_of_inclust_dist):.3f}.')
    print(f'Average distance to cluster center = '
      f'{count_clustering_scores(X, n_cls, model, mean_dist_to_center):.2f}.')


# Plot functions

def metric_score_results_for_models(X, label_list, score_fun):
    scores = []    
    for labels in label_list:
        wcss = score_fun(X, labels)
        scores.append(wcss)        
    return scores


def generate_model_labels(X, cluster_range, model, linkage_methods):
    labels_list = []    
    if model.__name__ == "KMeans":
        for linkage in linkage_methods:
            temp_list = []
            for k in cluster_range:
                model_instance = model(n_clusters=k, algorithm=linkage)
                labels = model_instance.fit_predict(X)
                temp_list.append(labels)
            labels_list.append(temp_list)
    elif model.__name__ == "AgglomerativeClustering":
        for linkage in linkage_methods:
            temp_list = []
            for k in cluster_range:
                model_instance = model(n_clusters=k, linkage=linkage)
                labels = model_instance.fit_predict(X)
                temp_list.append(labels)
            labels_list.append(temp_list)
    return labels_list

### KMeans

Na początku spójrzmy jak są dzielone na róźne ilości klastrów nasze obserwacje.

In [ ]:
def plot_kmeans_clusters(data, n_clusters):
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    kmeans.fit(data)
    y_kmeans = kmeans.predict(data)
    plt.scatter(data[:, 0], data[:, 1], c=y_kmeans, s=30, cmap='plasma')

    centers = kmeans.cluster_centers_
    plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.75)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.title(f'K-means clusters, k={n_clusters}')
    plt.show()

In [ ]:
k = 11

fig, axes = plt.subplots(k,2, figsize = (5*2, 10*k))
sdata = data.to_numpy()

algorithms_list = k * ['full', 'elkan']
n_clusters_list = [val for val in range(1,k+1,1) for _ in (0, 1)]

for algorithm, ax, n_cl, n in zip(algorithms_list, axes.flatten(), n_clusters_list, range(2*k)):
    model = KMeans(n_clusters=n_cl, algorithm=algorithms_list[n % 2])
    model.fit(sdata)
    y_predict = model.predict(sdata)
    ax.scatter(sdata[:, 0], sdata[:, 1], c=y_predict, s=20, cmap='plasma')
    centers = model.cluster_centers_
    ax.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.75)
    ax.set_aspect('equal', adjustable='box')
    ax.set_title(f'Num of clusters: {n_cl}, algorithm: {algorithms_list[n % 2]}')
plt.show()

In [ ]:
smaller_data = data.sample(frac=0.1).to_numpy()

algorithms_list = ['full', 'elkan']
cluster_range = [x for x in range(2, 12)]

metric_function_list_doubled = [val for val in metric_function_list for _ in (0, 1)]
k = len(metric_function_list_doubled)

fig, axes = plt.subplots(k//2,2, figsize = (2*10, 10*k//2))

label_list = generate_model_labels(smaller_data, cluster_range, KMeans, algorithms_list)
for fun, ax, n in zip(metric_function_list_doubled, axes.flatten(), range(k)):
    scores = metric_score_results_for_models(smaller_data, label_list[n%2], fun)
    ax.plot(cluster_range, scores, 'bx-')
    ax.set_xlabel('k')
    ax.set_ylabel(f'{fun.__name__} score')
    ax.set_title(f'KMeans with parameter (linkage/algorithm): {algorithms_list[n%2]}')

plt.show()

Jak widać odległości między punktami wewnątrz klastrów i między nimi naturalnie malała wraz ze wzrostem liczby podziałów. Co jednak ciekawe, dla k = 4, i algorytmu `full` mamy znacznie wyższą od reszty minimalną odległość między klastrami, co sugeruje, że wtedy są one najlepiej odseparowane.

In [ ]:
def count_wcss_scores(X, k_max, algorithm):
    #  WCSS = within-cluster sum of squares
    scores = []
    for k in range(1, k_max+1):
        kmeans = KMeans(n_clusters=k, random_state=0, algorithm=algorithm)
        kmeans.fit(X)
        wcss = kmeans.score(X) * -1 # score returns -WCSS
        scores.append(wcss)
    return scores

In [ ]:
fig, ax = plt.subplots(1,2, figsize = (20, 10))
wcss_vec = count_wcss_scores(data, 20, 'full')
x_ticks = list(range(1, len(wcss_vec) + 1))
ax[0].plot(x_ticks, wcss_vec, 'bx-')
ax[0].set_xlabel('k')
ax[0].set_ylabel('Within-cluster sum of squares')
ax[0].set_title('The Elbow Method showing the optimal k for full argorithm option')

wcss_vec = count_wcss_scores(data, 20, 'elkan')
x_ticks = list(range(1, len(wcss_vec) + 1))
ax[1].plot(x_ticks, wcss_vec, 'bx-')
ax[1].set_xlabel('k')
ax[1].set_ylabel('Within-cluster sum of squares')
ax[1].set_title('The Elbow Method showing the optimal k for elkan argorithm option')

plt.show()

Z powyższych dwóch wykresów widać, że w przypadku obu metod (dwa różne algorytmy użyte przy implementacji) optymalna liczba klastrów wynosi około 5-6. Należy tutaj pamiętać, że odczytywanie takich informacji z tego typu wykresów nie stanowi ścisłego dowodu, a jest pewnego rodzaju heurystyką.

Łącząc wnioski w poprzednich wykresów można wysunąć wnosek, że optymalna liczba klastrów wynosi bliżej 5. Jednocześnie warto zwrócić uwagę, że ta liczba może by także inna, bo w zależności od tego jak chcemy dzielić obserwacje, będziemy wykożystywać inne metryki. Wtedy to już nie ma gwarancji, że wyciągnięte wnioski będą analogiczne.

Mimo to, jeśli byłaby potrzeba podania kokretnej liczby podziałów, to odpowiedź brzmiałaby 5.

### AgglomerativeClustering

In [ ]:
def aglomerative_clusters_plot(X, n_clusters, ax):
    model = AgglomerativeClustering(n_clusters=n_clusters)
    y = model.fit_predict(X)
    plt.scatter(X[:, 0], X[:, 1], c=y, s=30, cmap='plasma', ax=ax)
    plt.show()

In [ ]:
smaller_data = data.sample(frac=0.1).to_numpy()

#### Linkage: Ward vs Complete

Na początku spójrzmy jak są dzielone na róźne ilości klastrów nasze obserwacje.

In [ ]:
k = 6

fig, axes = plt.subplots(k,2, figsize = (5*2, 10*k))


cluster_list_prep = [x for x in range(2,2*k+2,2) ]
cluster_list_prep = [*cluster_list_prep[:2], 5, *cluster_list_prep[2:]] 

linkage_methods = k * ['ward', 'complete']
n_clusters_list = [val for val in cluster_list_prep for _ in (0, 1)]

for linkage, ax, n_cl, n in zip(linkage_methods, axes.flatten(), n_clusters_list, range(12)):
    model = AgglomerativeClustering(n_clusters=n_cl, linkage=linkage_methods[n % 2])
    y = model.fit_predict(smaller_data)
    ax.scatter(smaller_data[:, 0], smaller_data[:, 1], c=y, s=30, cmap='plasma')
    ax.set_aspect('equal', adjustable='box')
    ax.set_title(f'Num of clusters: {n_cl}, linkage method: {linkage_methods[n % 2]}')
plt.show()

In [ ]:
# smaller_data = data.sample(frac=0.1).to_numpy()

linkage_list = ['ward', 'complete']
cluster_range = [x for x in range(2, 12, 2)]

metric_function_list_doubled = [val for val in metric_function_list for _ in (0, 1)]
k = len(metric_function_list_doubled)

fig, axes = plt.subplots(k//2,2, figsize = (2*10, 10*k//2))

label_list = generate_model_labels(smaller_data, cluster_range, AgglomerativeClustering, algorithms_list)
for fun, ax, n in zip(metric_function_list_doubled, axes.flatten(), range(k)):
    scores = metric_score_results_for_models(smaller_data, label_list[n%2], fun)
    ax.plot(cluster_range, scores, 'bx-')
    ax.set_xlabel('k')
    ax.set_ylabel(f'{fun.__name__} score')
    ax.set_title(f'AgglomerativeClustering with parameter (linkage/algorithm): {algorithms_list[n%2]}')

plt.show()

Porównując działanie algorytmu dla opcji `ward` i `complete` parametru linkage zachowują się dość podobnie, z wyjątkiem odchylenia standardowego między punktami w tym samym klastrze. Dla opcji `ward` jedynie dla 10 klastrów wartość ta się zmienia. Mówi nam to, że klastry są niezrównoważonych rozmiarów. W przypadku opcji `complete`, występuje tendencja do niezbalansowanych rozmiarów grup punktów, lecz wyjątkowo dla 4 klastrów, są one dobrze zbalansowane. Szczególnie widać to dobrze na mapach z zaznaczonymi klastrami.

#### Linkage: Average vs Single

In [ ]:
k=6

fig, axes = plt.subplots(6,2, figsize = (5*2, 10*k))

cluster_list_prep = [x for x in range(2,2*k+2,2) ]
cluster_list_prep = [*cluster_list_prep[:2], 5, *cluster_list_prep[2:]] 

linkage_methods = k * ['average', 'single']
n_clusters_list = [val for val in cluster_list_prep for _ in (0, 1)]

for linkage, ax, n_cl, n in zip(linkage_methods, axes.flatten(), n_clusters_list, range(12)):
    model = AgglomerativeClustering(n_clusters=n_cl, linkage=linkage_methods[n % 2])
    y = model.fit_predict(smaller_data)
    ax.scatter(smaller_data[:, 0], smaller_data[:, 1], c=y, s=30, cmap='plasma')
    ax.set_aspect('equal', adjustable='box')
    ax.set_title(f'Num of clusters: {n_cl}, linkage method: {linkage_methods[n % 2]}')
plt.show()

W przeciwieństwie do poprzenich metod łączenia klastrów, teraz widzimy wyraźne różnice w ich działaniu. 
Zastosowanie metody połączeń pojedyńczych, niesie za sobą to, że niezależnie od liczby klastrów, jeden z nich jest zawsze zdecydowanie większy od pozostałych.
Możemy więc wnioskować, że ta metoda nie dzieli danych na klastry o porównywalnych licznościach.
Metoda połączeń średnich działała w głównej mierze podobnie do wcześniejszych.

In [ ]:
# smaller_data = data.sample(frac=0.1).to_numpy()

linkage_list = ['average', 'single']
cluster_range = [x for x in range(2, 12, 2)]

metric_function_list_doubled = [val for val in metric_function_list for _ in (0, 1)]
k = len(metric_function_list_doubled)

fig, axes = plt.subplots(k//2,2, figsize = (2*10, 10*k//2))

label_list = generate_model_labels(smaller_data, cluster_range, AgglomerativeClustering, linkage_list)
for fun, ax, n in zip(metric_function_list_doubled, axes.flatten(), range(k)):
    scores = metric_score_results_for_models(smaller_data, label_list[n%2], fun)
    ax.plot(cluster_range, scores, 'bx-')
    ax.set_xlabel('k')
    ax.set_ylabel(f'{fun.__name__} score')
    ax.set_title(f'AgglomerativeClustering with parameter (linkage/algorithm): {linkage_list[n%2]}')

plt.show()

Większość metryk, dla obu metod zachowywała się podobnie, otrzymywany wartości stopniowo malały wraz z wrostem liczby klastrów.
Odchylenie standardowe wewnątrzklastrowe w przypadku metody połączeń średnich, z początku malało lecz ostatecznie zaczeło rosnąć, minimum zostało osiągnięte gdy liczba klastrów wynosiła 6.

## Podsumowanie



Przy używaniu `KMeans`, z metody łokcia dla wyszło nam, że optymalną liczba klastrów jest 5. Gdy pracowaliśmy z `AgglomerativeClustering` dla rożnych metod łączenia wnioski były podobne. Wyniki dla wielu metryk malały wraz ze wzrostem klastrów, a niektóre z nich miały najmniejsze wartości w okolicach liczb 4-6. Tak więc widzimy pewne wskazania na liczbę 5.  

Patrząc na wizualne podziały na klastry gdy było ich 5, łatwo znaleźć taki, który dokonuje podziału na regiony Wielkiej Brytanii.
Przykładowo podział dokonany metodą aglomeracyjną, wykorzystujący połączenia średnie wyraźnie wyodrębnił region `South West` razem z częścią Walii, Londyn wraz z otaczającymi go regionami, centralną i północną część Wielkiej Brytanii oraz Szkocję.
Gdy spojrzymy na mapę, taki podział jest zgodny z nasza intuicją, dlatego optymalną liczbą klastrów jest według nas 5.

## Walidacja

Walidujący: Daniel Tytkowski, Jan Skwarek

Sprawdzimy tylko co się stanie, jeśli przekształcimy współrzędne geograficzne na trójwymiarowe punkty na sferze i wtedy je sklastrujemy, używając SphericalKmeans.

In [ ]:
def cartesian_encoder(coord, r_E=6371):  #6371
    """Convert lat/lon to cartesian points on Earth's surface.

    Input
    -----
        coord : numpy 2darray (size=(N, 2))
        r_E : radius of Earth

    Output
    ------
        out : numpy 2darray (size=(N, 3))
    """
    lat , lon = np.deg2rad(coord[:,1]), np.deg2rad(coord[:,0])
    x = r_E * np.cos(lat) * np.cos(lon)
    y = r_E * np.sin(lon) * np.cos(lat)
    z = r_E * np.sin(lat)
    #return x, y, z

    return np.concatenate([x.reshape(-1, 1), y.reshape(-1, 1), z.reshape(-1, 1)], axis=1)

In [ ]:
df = pd.read_csv('/Users/danieltytkowski/Downloads/urbanGB/urbanGB.txt', header = None)
df = df.sample(frac=0.3)
df = np.array(df)

In [ ]:
df_sphere = cartesian_encoder(df)

In [ ]:
import plotly.express as px
def view3Dscatter(data,color=None):
    fig = px.scatter_3d(x=data[:,0], y=data[:,1], z=data[:,2], color=color)
    
    fig.show() 

In [ ]:
view3Dscatter(df_sphere)

In [ ]:
! pip install coclust
from coclust.clustering import SphericalKmeans

In [ ]:
sphereKmeans = KMeans(n_clusters=5)
sphereKmeans2 = SphericalKmeans(n_clusters=5)

In [ ]:
cols = sphereKmeans.fit_predict(df)

In [ ]:
sphereKmeans.score(df_sphere)

In [ ]:
colsClassicKMeansWithSpherical = sphereKmeans.fit_predict(df_sphere)

In [ ]:
sphereKmeans2.fit(df_sphere)
cols2=sphereKmeans2.row_labels_

In [ ]:
view3Dscatter(df_sphere,cols)

In [ ]:
view3Dscatter(df_sphere, cols2)

In [ ]:
view3Dscatter(df_sphere, colsClassicKMeansWithSpherical)

Podzial na klastry w 3 przypadkach: zwykły Kmeans+surowe wspolrzedne, spferyczny Kmeans+wspolrzednie trojwymiarowe i zwykly Kmeans+wspolrzedne trojwymiarowe roznia sie miedzy soba.

In [ ]:
def metrics_plots(max_k=10, X=None):

    score = []
    score_kmeans_s = []
    score_kmeans_c = []
    score_kmeans_d = []

    for k in range(2, max_k):
        kmeans = KMeans(n_clusters=k, random_state= 101)
        predictions = kmeans.fit_predict(X)
        # Calculate cluster validation metrics and append to lists of metrics
        score.append(kmeans.score(X))
        score_kmeans_s.append(silhouette_score(X, kmeans.labels_, metric='euclidean'))
        score_kmeans_c.append(calinski_harabasz_score(X, kmeans.labels_))
        score_kmeans_d.append(davies_bouldin_score(X, predictions))

    list_scores = [score, score_kmeans_s, score_kmeans_c, score_kmeans_d] 
    # Elbow Method plot
    list_title = ['Within-cluster sum of squares', 'Silhouette Score', 'Calinski Harabasz', 'Davies Bouldin'] 
    for i in range(len(list_scores)):
        x_ticks = list(range(2, len(list_scores[i]) + 2))
        plt.plot(x_ticks, list_scores[i], 'bx-')
        plt.xlabel('k')
        plt.ylabel(list_title[i])
        plt.title('Optimal k')
        plt.show()

In [ ]:
score = []
for x in range(2,10):
    model = KMeans(n_clusters=x)
    model.fit(df_sphere)
    score.append(model.score(df_sphere))
x_ticks = list(range(2, 10))
plt.plot(x_ticks, score, 'bx-')
plt.xlabel('k')
plt.ylabel('Within-cluster sum of squares')
plt.title('Optimal k')
plt.show()


Podobnie jak wczesniej klastrow powinno byc 4 lub 5

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
from sklearn.metrics import silhouette_score 
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics import davies_bouldin_score

Sprawdzmy jak poradzi sobie DBSCAN

In [ ]:
import sklearn

In [ ]:
minPts = 6
nbrs = sklearn.neighbors.NearestNeighbors(n_neighbors=minPts).fit(df_sphere)
distances, indices = nbrs.kneighbors(df_sphere)
distanceDec = sorted(distances[:,minPts-1], reverse=True)
fig = plt.figure(figsize=(9,6))
ax1 = fig.add_subplot(111)

plt.xlabel('Indeks punktu po sortowaniu')
plt.ylabel('Dystans od trzeciego najbliższego sąsiada')
ax1.plot(list(range(1,df_sphere.shape[0]+1)), distanceDec)

plt.xscale('log')
plt.grid(axis='y')

plt.show()

In [ ]:
sphereDBCAN = DBSCAN(eps=4)

In [ ]:
dbCols = sphereDBCAN.fit_predict(df_sphere)

In [ ]:
view3Dscatter(df_sphere, dbCols)

Praca wydaje się być w gruncie rzeczy bardzo dobra. Jedyny błąd to nie wzięcię pod uwagę, że punkty ze zbioru to punkty na sferze, a nie na płaszczyźnie (sprawdzimy czy robi to jakąś różnicę). Praca posiada podsumowanie, wiele wykresów, ciekawych wizualizacji. Metody zaproponowane przez zespół budujący są sprawdzane wieloma metrykami. Jest tylko kilka drobnych uwag, które postaramy się wypunktować:

1. Mało komentarzy - mogłoby się ich pojawić troszkę więcej. Szczególnie mowa tu o komentarzach przy funkcjach.
2. Zdecydowana większość kodu skopiowana z laboratoryjnego pliku. Można było o tym wspomnieć.
3. Brak informacji dla walidujących przed potencjalnie długo wykonującymi się funkjcami, czy też takimi, które bardzo obciążają RAM.
4. Brak argumentacji dlaczego budujący decydują się akurat na taki wybór modeli lub chociażby informacji, że wybrane zostały one losowo.
5. Brak informacji, która metoda klastreryzacji okazała się finalnie lepsza i dlaczego (samo wyciągnięcie wniosków odnośnie liczby klastrów zostało przeprowadzone dobrze).
6. Metody są sprawdzane na 10% próbce, która jest inna dla każdej metody - nie jest to do końca błędne, ale można byłoby zrobić wizualizacje również dla tej samej próbki.

Podsumowując, praca jest naprawdę bardzo solidnie wykonana. Nie wykryliśmy żadnych poważnych błędów. Analizy są wnikliwe, a wizualizacje czytelne. Widać, że budujący opanowali dobrze metody klasteryzacji i potrafią szukać skupień w tego typu zbiorach.